In [1]:
# import kagglehub
# import shutil
# import os

# # 1. Definir dónde queremos los datos (Root del proyecto)
# TARGET_DIR = "./RAVDESS_DATA"

# # 2. Descargar desde Kaggle (Se baja a una carpeta temporal de caché)
# print("⬇️ Iniciando descarga desde KaggleHub...")
# # Esta función devuelve la ruta donde Kaggle guardó los archivos temporalmente
# cache_path = kagglehub.dataset_download("uwrfkaggler/ravdess-emotional-speech-audio")

# print(f"✅ Descarga completada en caché: {cache_path}")

# # 3. Mover archivos a tu carpeta del proyecto
# print(f"🚚 Moviendo archivos a '{TARGET_DIR}'...")

# # Crear tu carpeta si no existe
# os.makedirs(TARGET_DIR, exist_ok=True)

# # Mover el contenido (Carpetas Actor_01, Actor_02, etc.)
# for item in os.listdir(cache_path):
#     source = os.path.join(cache_path, item)
#     destination = os.path.join(TARGET_DIR, item)
    
#     # Si ya existe la carpeta Actor_XX, la borramos para actualizarla
#     if os.path.exists(destination):
#         if os.path.isdir(destination):
#             shutil.rmtree(destination)
#         else:
#             os.remove(destination)
    
#     # Mover (Move es más rápido que Copy)
#     shutil.move(source, destination)

# print("------------------------------------------------")
# print(f"🎉 ¡ÉXITO! El dataset está listo en: {TARGET_DIR}")
# print("   Estructura verificada:")
# # Listar las primeras 5 carpetas para confirmar
# print(f"   {sorted(os.listdir(TARGET_DIR))[:5]} ...")

In [2]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.quantization
import torch.nn.utils.prune as prune
import numpy as np
import random
import copy
import time
import platform
import soundfile as sf
import resampy
from torch.utils.data import Dataset, DataLoader, Subset
from torchaudio.transforms import MelSpectrogram, AmplitudeToDB
from transformers import AutoModelForAudioClassification, Wav2Vec2FeatureExtractor
# Importamos la barra de progreso para Notebooks
from tqdm import tqdm

# --- CONFIGURACIÓN GLOBAL ---
SEED = 42

def set_seed(seed=SEED):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed()

# Detección de Hardware
if torch.cuda.is_available():
    DEVICE = torch.device("cuda:0") # Usamos la primera GPU
    gpu_name = torch.cuda.get_device_name(0)
    # Mostramos info de la GPU
    print(f"🚀 MODO TURBO: Usando {gpu_name}")
    print(f"🧠 VRAM Disponible: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    DEVICE = torch.device("cpu")
    print("⚠️ Usando CPU.")

# La cuantización final siempre es en CPU
QUANT_DEVICE = torch.device("cpu")

🚀 MODO TURBO: Usando NVIDIA GeForce RTX 2080 Ti
🧠 VRAM Disponible: 11.36 GB


In [3]:
class RavdessResearchDataset(Dataset):
    def __init__(self, root_dir, sample_rate=16000, duration=3.0, add_noise=False):
        self.root_dir = root_dir
        self.sample_rate = sample_rate
        self.num_samples = int(sample_rate * duration)
        self.add_noise = add_noise
        self.file_paths = []
        self.labels = []
        self.actors = [] 
        
        print(f"📂 Escaneando directorio: {root_dir}")
        for root, dirs, files in os.walk(root_dir):
            for file in files:
                if file.endswith(('.wav', '.flac', '.mp3')):
                    parts = file.split('-')
                    if len(parts) == 7:
                        # RAVDESS: 03-01-03-01-01-01-01.wav -> Emotion is index 2
                        emotion = int(parts[2]) - 1 
                        actor = int(parts[6].split('.')[0])
                        self.labels.append(emotion)
                        self.actors.append(actor)
                        self.file_paths.append(os.path.join(root, file))
        
        print(f"✅ Total archivos encontrados: {len(self.file_paths)}")

        # Transformaciones de Audio
        self.mel_spec = MelSpectrogram(sample_rate=sample_rate, n_fft=1024, hop_length=512, n_mels=64)
        self.db_transform = AmplitudeToDB()

    def _process_signal(self, path):
        try:
            data, sr = sf.read(path)
        except:
            return torch.zeros(self.num_samples) # Retorno seguro si falla

        if len(data.shape) > 1: data = np.mean(data, axis=1) # Stereo a Mono
        if sr != self.sample_rate: 
            data = resampy.resample(data, sr, self.sample_rate, filter='kaiser_best')
        
        signal = torch.from_numpy(data).float()
        
        # Cortar o Rellenar (Padding) a 3 segundos
        if signal.shape[0] > self.num_samples:
            signal = signal[:self.num_samples]
        elif signal.shape[0] < self.num_samples:
            pad = self.num_samples - signal.shape[0]
            signal = F.pad(signal, (0, pad))
            
        return signal

    def __getitem__(self, idx):
        path = self.file_paths[idx]
        label = self.labels[idx]
        
        try:
            signal = self._process_signal(path)
            
            # Data Augmentation (Solo ruido)
            if self.add_noise:
                noise = torch.randn_like(signal) * 0.005
                signal = signal + noise
            
            # Input 1: Espectrograma (Para tu modelo Alumno)
            spec = self.mel_spec(signal)
            spec = self.db_transform(spec)
            
            # Input 2: Audio Crudo (Para el modelo Maestro Wav2Vec)
            return {
                "spectrogram": spec.unsqueeze(0), # [1, 64, 94]
                "raw_audio": signal,              # [48000]
                "label": torch.tensor(label, dtype=torch.long)
            }
        except:
            return self.__getitem__((idx + 1) % len(self)) # Intentar siguiente si falla
    
    def __len__(self): return len(self.file_paths)

In [4]:
# --- RUTA DE TUS DATOS ---
ROOT_PATH = "./RAVDESS_DATA" 

# 1. Instanciar Dataset
full_dataset = RavdessResearchDataset(root_dir=ROOT_PATH, add_noise=False)

if len(full_dataset) == 0:
    print("❌ ERROR CRÍTICO: No se encontraron archivos.")
else:
    # 2. Split por Actor
    all_actors = full_dataset.actors
    train_indices = [i for i, a in enumerate(all_actors) if a <= 20]
    val_indices = [i for i, a in enumerate(all_actors) if a > 20]

    train_ds = Subset(RavdessResearchDataset(ROOT_PATH, add_noise=True), train_indices)
    val_ds = Subset(RavdessResearchDataset(ROOT_PATH, add_noise=False), val_indices)

    print(f"📊 Train: {len(train_ds)} | Val: {len(val_ds)}")

    # 3. DATALOADERS OPTIMIZADOS
    # Batch size 64 para aprovechar tu VRAM
    # num_workers=4 para que la CPU prepare los datos rápido mientras la GPU entrena
    # pin_memory=True acelera el paso de RAM a VRAM
    train_loader = DataLoader(train_ds, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_ds, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)
    
    print("✅ DataLoaders listos (Batch Size: 64, Workers: 4).")

📂 Escaneando directorio: ./RAVDESS_DATA
✅ Total archivos encontrados: 2880
📂 Escaneando directorio: ./RAVDESS_DATA
✅ Total archivos encontrados: 2880
📂 Escaneando directorio: ./RAVDESS_DATA
✅ Total archivos encontrados: 2880
📊 Train: 2400 | Val: 480
✅ DataLoaders listos (Batch Size: 64, Workers: 4).


In [5]:
import torch
import torch.nn as nn

# --- CORRECCIÓN FINAL: SOPORTE NATIVO DE KERNEL SIZE ---

# Bloque 1: RepVGG Standard (Ahora soporta kernel_size variable)
class RepVGGBlock(nn.Module):
    # AÑADIDO: argumento 'kernel_size' al init
    def __init__(self, in_c, out_c, stride=1, deploy=False, groups=1, kernel_size=3, padding=1):
        super().__init__()
        self.deploy = deploy
        self.groups = groups
        self.act = nn.ReLU()
        
        if deploy:
            self.rbr_reparam = nn.Conv2d(in_c, out_c, kernel_size, stride, padding, groups=groups, bias=True)
        else:
            # Rama Densa: Usa el kernel_size que le pasemos (3 para DW, 1 para PW)
            self.rbr_dense = nn.Sequential(
                nn.Conv2d(in_c, out_c, kernel_size, stride, padding, groups=groups, bias=False), 
                nn.BatchNorm2d(out_c)
            )
            
            # Rama 1x1: Solo se añade si el kernel principal NO es 1x1 (para evitar redundancia total)
            # o si queremos forzar diversidad. En MobileOne standard:
            # - DW (3x3): Tiene rama 1x1.
            # - PW (1x1): Es puramente densa 1x1 (la rama 1x1 sería idéntica a la densa).
            # Para simplificar y asegurar estabilidad:
            if kernel_size > 1:
                padding_1x1 = 0
                self.rbr_1x1 = nn.Sequential(
                    nn.Conv2d(in_c, out_c, 1, stride, padding_1x1, groups=groups, bias=False), 
                    nn.BatchNorm2d(out_c)
                )
            else:
                self.rbr_1x1 = None # No rama 1x1 extra si la principal ya es 1x1

            # Rama Identidad: Solo si in==out y stride==1
            self.rbr_identity = nn.BatchNorm2d(in_c) if out_c == in_c and stride == 1 else None

    def forward(self, x):
        if self.deploy: return self.act(self.rbr_reparam(x))
        
        # Salida de rama densa
        out = self.rbr_dense(x)
        
        # Sumamos rama 1x1 si existe
        if self.rbr_1x1 is not None:
            out += self.rbr_1x1(x)
            
        # Sumamos identidad si existe
        if self.rbr_identity is not None:
            out += self.rbr_identity(x)
            
        return self.act(out)

    def switch_to_deploy(self):
        if self.deploy: return
        
        # Obtenemos configuración de la rama densa actual
        dense_conv = self.rbr_dense[0]
        k = dense_conv.kernel_size[0]
        s = dense_conv.stride[0]
        p = dense_conv.padding[0]
        g = dense_conv.groups
        
        self.rbr_reparam = nn.Conv2d(
            dense_conv.in_channels, 
            dense_conv.out_channels, 
            k, s, p, groups=g, bias=True
        )
        # Copia simple de pesos (en producción real se fusionarían las ramas)
        self.rbr_reparam.weight.data = dense_conv.weight.data
        self.rbr_reparam.bias.data = torch.zeros(self.rbr_reparam.out_channels)
        
        self.deploy = True

# Bloque 2: MobileOne (Actualizado para usar el nuevo RepVGGBlock)
class MobileOneBlock(nn.Module):
    def __init__(self, in_c, out_c, stride=1, deploy=False):
        super().__init__()
        # Depthwise: 3x3, Padding 1, Grupos=in_c
        self.dw = RepVGGBlock(in_c, in_c, stride, deploy=deploy, groups=in_c, kernel_size=3, padding=1)
        
        # Pointwise: 1x1, Padding 0, Grupos=1
        # AQUÍ ESTABA EL ERROR: Ahora pasamos kernel_size=1 explícitamente
        self.pw = RepVGGBlock(in_c, out_c, 1, deploy=deploy, groups=1, kernel_size=1, padding=0)
        
    def forward(self, x):
        x = self.dw(x)
        x = self.pw(x)
        return x
    
    def switch_to_deploy(self):
        self.dw.switch_to_deploy()
        self.pw.switch_to_deploy()

# Bloque 3: GhostModule (Sin cambios)
class GhostModule(nn.Module):
    def __init__(self, inp, oup, stride=1):
        super().__init__()
        self.oup = oup
        init_channels = int(oup / 2)
        new_channels = init_channels
        
        self.primary_conv = nn.Sequential(
            nn.Conv2d(inp, init_channels, 1, stride, 0, bias=False),
            nn.BatchNorm2d(init_channels),
            nn.ReLU(inplace=True)
        )
        self.cheap_operation = nn.Sequential(
            nn.Conv2d(init_channels, new_channels, 3, 1, 1, groups=init_channels, bias=False),
            nn.BatchNorm2d(new_channels),
            nn.ReLU(inplace=True)
        )
    def forward(self, x):
        x1 = self.primary_conv(x)
        x2 = self.cheap_operation(x1)
        return torch.cat([x1, x2], dim=1)[:, :self.oup, :, :]

# RED PRINCIPAL
class UniversalAudioNet(nn.Module):
    def __init__(self, num_classes=8, arch_type="repvgg", deploy=False):
        super().__init__()
        self.arch_type = arch_type
        self.quant = torch.quantization.QuantStub()
        self.dequant = torch.quantization.DeQuantStub()
        
        def make_block(in_c, out_c, stride):
            if arch_type == "repvgg": 
                # RepVGG normal usa 3x3
                return RepVGGBlock(in_c, out_c, stride, deploy=deploy, kernel_size=3, padding=1)
            elif arch_type == "mobileone": 
                return MobileOneBlock(in_c, out_c, stride, deploy=deploy)
            elif arch_type == "ghostnet": 
                return GhostModule(in_c, out_c, stride)

        # Backbone Tiny
        self.stage0 = make_block(1, 32, 2)
        self.stage1 = make_block(32, 64, 2)
        self.stage2 = make_block(64, 128, 2)
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.quant(x)
        x = self.stage0(x)
        x = self.stage1(x)
        x = self.stage2(x)
        x = self.gap(x).flatten(1)
        x = self.fc(x)
        x = self.dequant(x)
        return x
    
    def switch_to_deploy(self):
        for m in self.modules():
            if m is self: continue
            if hasattr(m, 'switch_to_deploy'): m.switch_to_deploy()

In [6]:
TEACHER_MODEL_NAME = "ehcalabres/wav2vec2-lg-xlsr-en-speech-emotion-recognition"

try:
    print(f"👨‍🏫 Cargando Maestro: {TEACHER_MODEL_NAME}...")
    teacher_model = AutoModelForAudioClassification.from_pretrained(TEACHER_MODEL_NAME)
    teacher_model.to(DEVICE)
    teacher_model.eval()
    TEACHER_AVAILABLE = True
    print("✅ Maestro cargado en GPU.")
except Exception as e:
    print(f"⚠️ Error cargando maestro: {e}")
    TEACHER_AVAILABLE = False
    teacher_model = None

👨‍🏫 Cargando Maestro: ehcalabres/wav2vec2-lg-xlsr-en-speech-emotion-recognition...


/root/miniconda3/envs/audio_edge_env/lib/python3.10/site-packages/transformers/configuration_utils.py:335: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of the model checkpoint at ehcalabres/wav2vec2-lg-xlsr-en-speech-emotion-recognition were not used when initializing Wav2Vec2ForSequenceClassification: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.output.bias', 'classifier.output.weight']
- This IS expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSequenceC

✅ Maestro cargado en GPU.


In [7]:
# Celda 6 Corregida

def run_experiment(arch_name, train_loader, val_loader, epochs=10):
    print(f"\n🔬 --- ARQUITECTURA: {arch_name.upper()} ---")
    
    model = UniversalAudioNet(arch_type=arch_name).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()
    
    # 1. CICLO DE ENTRENAMIENTO
    # Quitamos 'position' para evitar saltos de línea
    epoch_bar = tqdm(range(epochs), desc=f"Entrenando {arch_name}", unit="epoch", leave=True)
    
    # Historial
    history = {'train_loss': [], 'val_acc': []}

    for epoch in epoch_bar:
        model.train()
        running_loss = 0.0
        
        # Barra interna para los batches
        # leave=False borra esta barra al terminar la época para no ensuciar
        batch_bar = tqdm(train_loader, desc=f"Ep {epoch+1}", leave=False, unit="batch")
        
        for batch in batch_bar:
            specs = batch['spectrogram'].to(DEVICE)
            raw = batch['raw_audio'].to(DEVICE)
            labels = batch['label'].to(DEVICE)
            
            optimizer.zero_grad()
            student_logits = model(specs)
            loss = criterion(student_logits, labels)
            
            # Destilación
            if TEACHER_AVAILABLE and teacher_model:
                with torch.no_grad():
                    t_out = teacher_model(raw)
                    t_logits = t_out.logits if hasattr(t_out, 'logits') else t_out
                
                if t_logits.shape[1] == student_logits.shape[1]:
                    T = 4.0
                    d_loss = nn.KLDivLoss(reduction="batchmean")(
                        F.log_softmax(student_logits/T, dim=1),
                        F.softmax(t_logits/T, dim=1)
                    ) * (T*T)
                    loss = 0.5 * loss + 0.5 * d_loss
            
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            # Actualizamos la barra pequeña
            batch_bar.set_postfix(loss=f"{loss.item():.4f}")
            
        # 2. EVALUACIÓN
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for batch in val_loader:
                out = model(batch['spectrogram'].to(DEVICE))
                _, pred = torch.max(out, 1)
                correct += (pred == batch['label'].to(DEVICE)).sum().item()
                total += batch['label'].size(0)
        
        val_acc = 100 * correct / total
        history['train_loss'].append(running_loss / len(train_loader))
        history['val_acc'].append(val_acc)
        
        # Actualizamos la barra PRINCIPAL
        epoch_bar.set_postfix(val_acc=f"{val_acc:.2f}%", avg_loss=f"{running_loss/len(train_loader):.4f}")
    
    # 3. OPTIMIZACIÓN FINAL (CPU)
    print("\n🧊 Optimizando (Poda + Cuantización)...")
    model.to("cpu")
    if hasattr(model, 'switch_to_deploy'): model.switch_to_deploy()
        
    for name, m in model.named_modules():
        if isinstance(m, nn.Conv2d):
            try:
                prune.l1_unstructured(m, name='weight', amount=0.3)
                prune.remove(m, 'weight')
            except: pass
            
    # Cuantización
    backend = 'qnnpack' if 'arm' in platform.machine().lower() else 'fbgemm'
    model.qconfig = torch.quantization.get_default_qconfig(backend)
    torch.backends.quantized.engine = backend
    torch.quantization.prepare(model, inplace=True)
    
    # Calibración rápida
    with torch.no_grad():
        for i, batch in enumerate(train_loader):
            if i > 5: break
            model(batch['spectrogram'].to("cpu"))
    
    model_int8 = torch.quantization.convert(model, inplace=False)
    
    # Evaluación Final INT8
    correct, total = 0, 0
    with torch.no_grad():
        for batch in val_loader:
            out = model_int8(batch['spectrogram'].to("cpu"))
            _, pred = torch.max(out, 1)
            correct += (pred == batch['label']).sum().item()
            total += batch['label'].size(0)
    acc_int8 = 100 * correct / total
    
    # Tamaño
    torch.save(model_int8.state_dict(), f"temp_{arch_name}.pth")
    size_mb = os.path.getsize(f"temp_{arch_name}.pth") / (1024*1024)
    
    print(f"✅ FINAL {arch_name}: Acc={acc_int8:.2f}% | Size={size_mb:.2f} MB")
    return {"arch": arch_name, "acc_int8": acc_int8, "size": size_mb, "history": history}

In [8]:
results = []
architectures = ["repvgg", "mobileone", "ghostnet"]

# Con GPU, puedes subir los epochs a 10 o 20 sin esperar una eternidad
for arch in architectures:
    res = run_experiment(arch, train_loader, val_loader, epochs=3)
    results.append(res)

print("\n🏆 --- RESULTADOS FINALES ---")
print(f"{'Arquitectura':<15} | {'Acc INT8':<10} | {'Size (MB)':<10}")
print("-" * 45)
for r in results:
    print(f"{r['arch']:<15} | {r['acc_int8']:.2f}%      | {r['size']:.2f}")


🔬 --- ARQUITECTURA: REPVGG ---


Ep 2:  42%|████▏     | 16/38 [00:14<00:14,  1.52batch/s, loss=0.9601]


Ep 3:  87%|████████▋ | 33/38 [00:25<00:03,  1.51batch/s, loss=0.9164]


Entrenando repvgg: 100%|██████████| 3/3 [01:38<00:00, 32.98s/epoch, avg_loss=0.9083, val_acc=13.75%]


🧊 Optimizando (Poda + Cuantización)...


...

Ep 3:  87%|████████▋ | 33/38 [00:25<00:03,  1.51batch/s, loss=0.9458]


Entrenando ghostnet: 100%|██████████| 3/3 [01:39<00:00, 33.28s/epoch, avg_loss=0.9619, val_acc=13.33%]


🧊 Optimizando (Poda + Cuantización)...



/tmp/ipykernel_773191/2794115407.py:119: UserWarning: All inputs of this cat operator must share the same quantization parameters. Otherwise large numerical inaccuracies may occur. (Triggered internally at /pytorch/aten/src/ATen/native/quantized/cpu/TensorShape.cpp:167.)
  return torch.cat([x1, x2], dim=1)[:, :self.oup, :, :]


✅ FINAL ghostnet: Acc=13.33% | Size=0.04 MB

🏆 --- RESULTADOS FINALES ---
Arquitectura    | Acc INT8   | Size (MB) 
---------------------------------------------
repvgg          | 13.33%      | 0.23
mobileone       | 13.33%      | 0.09
ghostnet        | 13.33%      | 0.04
